In [1]:
# Paso 1.
# Modifica las variables de WRF para poder hacer el mapeo de cuantiles.

import numpy as np
import pandas as pd

import xarray as xr
import os

# Inicializamos el dashboard de cómputo distribuido.
from dask.distributed import Client
c_lat = 1
c_lon = 1
Client( n_workers = 15, threads_per_worker = 5, memory_limit = "500MB" )

distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-283kaj_7', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 75,Total memory: 6.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55304,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 75
Started: Just now,Total memory: 6.98 GiB
Comm: tcp://127.0.0.1:55369,Total threads: 5
Dashboard: http://127.0.0.1:55371/status,Memory: 476.84 MiB
Nanny: tcp://127.0.0.1:55316,


In [2]:
# Cargamos el archivo.
path_d = "../data/WRF/"
path_r = "../temp/quantile_prep/"
path_v = "../temp/quantile_vars/"

files = os.listdir(path_d)
files.sort()
if ".DS_Store" in files: files.remove(".DS_Store")
for f in files:
    print(f"{f[-5:-3]}", end=",")
    with xr.open_dataset( path_d + f, chunks = {
        "south_north": c_lat, "west_east": c_lon } ) as ds:

        # Creamos las variables.
        ds["Wind Speed"] = np.sqrt( np.square(ds["U10"])
            + np.square(ds["V10"]) )
        ds["Wind Direction"] = ( np.arctan2(ds["V10"], ds["U10"])
            * 180/np.pi - 90 )
        ds["Wind Direction"] = ds["Wind Direction"].where(
            ds["Wind Direction"] > 0, ds["Wind Direction"] + 360 )
        ds = ds.drop_vars( ["U10", "V10"] )
        ds["T C"] = ds["T2"] - 273.15
        ds["Pvs"] = 611.21*np.exp( ( 18.678 - ds["T C"]/234.5 )
            * ( ds["T C"]/(257.14+ds["T C"]) ) )
        ds = ds.drop( "T C" )
        ds["Q2"] = ds["Q2"].where( ds["Q2"] > 0.0001, 0.0001 )
        ds["Relative Humidity"] = ( 100 * ds["Q2"] * 461.4/286.9
            * ( ds["PSFC"]/ds["Pvs"] - 1 ) )
        ds["Relative Humidity"] = ds["Relative Humidity"].where(
            ds["Relative Humidity"] < 100, 100 )
        ds = ds.drop_vars( ["Q2", "Pvs"] )

        # Reordenamos el Dataset.
        ds["Wind Speed"] = ds["Wind Speed"].assign_attrs( units = "m/s" )
        ds["Relative Humidity"] = ds[ "Relative Humidity"
            ].assign_attrs( units = "%" )
        ds["Wind Direction"] = ds ["Wind Direction"
            ].assign_attrs( units = "degrees" )
        ds = ds.rename_vars( { "XTIME": "time", "XLONG": "lon", "XLAT": "lat",
            "T2": "Temperature", "PSFC": "Pressure", "SWDOWN": "GHI" } )
        ds["lon"] = ds["lon"].isel({"south_north": 0})
        ds["lat"] = ds["lat"].isel({"west_east": 0})
        ds = ds.swap_dims( {"south_north": "lat",
            "west_east": "lon", "XTIME": "time"} )

        # Guardamos el archivo.
        # Esta línea arranca el cómputo distribuido.
        ds.to_netcdf( path_r + f, mode = "w" )

        vars = ["Temperature", "Pressure", "Relative Humidity",
            "Wind Speed", "Wind Direction", "GHI"]
        for v in vars:
            ds[v].to_netcdf( path_v + v + "/" + f, mode = "w" )

90,91,92,93,94,95,96,97,98,99,